In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd.functional import jacobian
from functorch import vmap, jacrev
import matplotlib.pyplot as plt

In [2]:
# ------------------------------
# Set up the device: GPU if available, otherwise CPU
# ------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ------------------------------
# Define ODE constants and the ODE function φ on the selected device
# ------------------------------
D = torch.tensor([-9.54, -8.16, -4.26, -11.43], dtype=torch.float32, device=device)
A = torch.tensor(
    [3.18, 2.72, 1.42, 3.81], dtype=torch.float32, device=device
)  # shape: (4,)
b = 7.81  # scalar


def phi(y):
    """
    Given y (a tensor of shape (5,)), compute:
      x = y[:4]
      u = y[4]
      m = max(0, u + dot(A, x) - b)
    and return
      [ -(D + A*m); m - u ] as a tensor of shape (5,)
    """
    x = y[:4]
    u = y[4]
    m = torch.clamp(u + torch.dot(A, x) - b, min=0.0)
    top = -(D + A * m)
    bottom = m - u
    return torch.cat((top, bottom.unsqueeze(0)))  # resulting shape: (5,)

Using device: cuda


In [3]:
# xx = torch.ones((5,), device=device).squeeze(0)
# phi(xx)

In [4]:
# ------------------------------
# Define the PINN model (moved to GPU)
# ------------------------------
class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        # Two-layer network: input dimension 1 -> 100 -> 5
        self.fc1 = nn.Linear(1, 100)
        self.fc2 = nn.Linear(100, 5)
        self.activation = nn.Tanh()

    def forward(self, t):
        """
        Forward pass for a scalar (or batch) time input.
        We assume t is a tensor of shape (N, 1) or a scalar tensor.
        The network output is modulated as:
            ŷ(t) = (1 - exp(-t)) * NN(t)
        to enforce ŷ(0) = 0.
        """
        if t.dim() == 0:
            t = t.unsqueeze(0)
        x = self.activation(self.fc1(t))
        out = self.fc2(x)
        return (1 - torch.exp(-t)) * out

In [5]:
# Instantiate and move the model to the device
model = PINN().to(device)

In [6]:
# ------------------------------
# Set up collocation points on the device
# ------------------------------
# 100 points uniformly in [0,10]
ts = torch.linspace(0, 10, 100, dtype=torch.float32, device=device)  # shape (100,)


# ------------------------------
# Define the vectorized loss function using functorch
# ------------------------------
def compute_loss_vectorized():
    # ts: shape (N,) ; we need to work with scalar inputs, so we keep ts as a 1D tensor.
    # Evaluate the PINN on all collocation points.
    # The model expects input shape (N,1), so unsqueeze ts.
    ts_var = ts.clone().detach().requires_grad_(True)  # shape (N,)
    y_hat = model(ts_var.unsqueeze(1))  # shape: (N, 5)

    # Define a function that maps a scalar t to the model output (a vector of shape (5,))
    def model_single(t):
        # t is a scalar; model expects shape (1,1)
        return model(t.unsqueeze(0)).squeeze(0)

    # Compute the derivative dy/dt for each scalar time t using vectorized jacobian.
    # jacrev computes the Jacobian of model_single at a scalar t (output shape: (5,))
    dy_dt = torch.vmap(torch.func.jacrev(model_single))(ts_var)  # shape: (N, 5)

    # Vectorize phi over the batch dimension.
    phi_y = torch.vmap(phi)(y_hat)  # shape: (N, 5)

    # Compute the residuals at each collocation point.
    residuals = dy_dt - phi_y  # shape: (N, 5)
    # Compute the mean squared residual over the collocation points.
    loss = torch.mean(torch.sum(residuals**2, dim=1))
    return loss

In [7]:
# compute_loss_vectorized()

In [8]:
# ------------------------------
# Training Loop using Adam (lr = 0.001) on the GPU
# ------------------------------
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 5000

print("Starting training...")
for epoch in range(1, epochs + 1):
    optimizer.zero_grad()
    loss_val = compute_loss_vectorized()
    loss_val.backward()
    optimizer.step()

    # Print loss every 100 epochs.
    if epoch % 10 == 0:
        print(f"Epoch {epoch}: Loss = {loss_val.item():.6f}")

    # Stop if loss is below the threshold.
    if loss_val.item() < 1e-6:
        print(f"Stopping training at epoch {epoch} with loss = {loss_val.item():.6f}")
        break

print("Training complete.\n")

# ------------------------------
# Evaluate the trained model at select time points
# ------------------------------
test_times = [0.0, 2.5, 5.0, 7.5, 10.0]
for t in test_times:
    t_tensor = torch.tensor(t, dtype=torch.float32, device=device, requires_grad=True)
    y_pred = model(t_tensor)
    print(f"t = {t:4.1f}, ŷ(t) = {y_pred.detach().cpu().numpy().flatten()}")

Starting training...
Epoch 10: Loss = 98.145134
Epoch 20: Loss = 92.299706
Epoch 30: Loss = 74.305138
Epoch 40: Loss = 60.802822
Epoch 50: Loss = 53.295628
Epoch 60: Loss = 50.553516
Epoch 70: Loss = 48.616375
Epoch 80: Loss = 46.806499
Epoch 90: Loss = 45.250790
Epoch 100: Loss = 43.746441
Epoch 110: Loss = 42.129402
Epoch 120: Loss = 40.522514
Epoch 130: Loss = 39.093140
Epoch 140: Loss = 37.805561
Epoch 150: Loss = 36.335915
Epoch 160: Loss = 35.016201
Epoch 170: Loss = 33.885036
Epoch 180: Loss = 32.888996
Epoch 190: Loss = 31.708719
Epoch 200: Loss = 30.659184
Epoch 210: Loss = 29.773582
Epoch 220: Loss = 29.025282
Epoch 230: Loss = 28.393898
Epoch 240: Loss = 27.548117
Epoch 250: Loss = 26.784922
Epoch 260: Loss = 26.136253
Epoch 270: Loss = 25.577135
Epoch 280: Loss = 25.101414
Epoch 290: Loss = 24.691139
Epoch 300: Loss = 24.309763
Epoch 310: Loss = 23.718740
Epoch 320: Loss = 23.181992
Epoch 330: Loss = 22.695177
Epoch 340: Loss = 22.261328
Epoch 350: Loss = 21.877590
Epoch 36

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set a seaborn theme
sns.set_theme(style="darkgrid")

# Create a figure
plt.figure(figsize=(10, 6))

# # Plot each solution component with seaborn's lineplot
# for i in range(5):
#     sns.lineplot(x=t_plot_np, y=y_plot[:, i], label=f'y[{i}]')

# plt.xlabel('t')
# plt.ylabel('y')
# plt.title('PINN Solution Components over [0,10]')
# plt.legend()
# plt.show()

<Figure size 1000x600 with 0 Axes>

In [9]:
t_ = torch.tensor(10.2, dtype=torch.float32, device=device, requires_grad=True)
y_ = model(t_)
y_

tensor([0.6711, 0.8113, 0.3215, 0.7965, 2.9876], device='cuda:0',
       grad_fn=<MulBackward0>)

In [1]:
# ------------------------------
# Evaluate the trained model at select time points and visualize each solution component.
# ------------------------------
# Create a fine grid over [0,10]
t_plot = torch.linspace(0, 10, 200, dtype=torch.float32, device=device)  # shape: (200,)
# Evaluate the model on this grid. (Ensure correct shape by unsqueezing.)
y_plot = model(t_plot.unsqueeze(1))  # shape: (200, 5)
y_plot = y_plot.detach().cpu().numpy()
t_plot_np = t_plot.detach().cpu().numpy()

# Create a plot for each component y[0]...y[4]
plt.figure(figsize=(10, 8))
for i in range(5):
    plt.plot(t_plot_np, y_plot[:, i], label=f"y[{i}]")
plt.xlabel("t")
plt.ylabel("y")
plt.title("PINN Solution Components over [0,10]")
plt.legend()
plt.grid(True)
plt.show()

NameError: name 'torch' is not defined